In [144]:
import scipy

In [132]:
size=10000
#population = np.random.choice(['a','b','c','d'], replace=True, size=size)#.astype(np.int64)
population = np.random.normal(50,8, size=size)#.astype(np.int64)
population

array([56.18194637, 63.76595351, 51.79716109, ..., 45.78595921,
       38.42408662, 61.23562377])

In [133]:
population.mean()

50.01587782877474

In [134]:
unique, counts = np.unique(population, return_counts=True)

In [135]:
B = 10000

In [147]:
samples_counts = scipy.stats.poisson.rvs(mu=1, size=size*B)

In [148]:
samples_counts = np.random.poisson(1, size*B)
samples_counts

array([0, 0, 1, ..., 0, 2, 0])

In [149]:
samples_counts_matrix = samples_counts.reshape(B,-1)
samples_counts_matrix

array([[0, 0, 1, ..., 0, 1, 0],
       [1, 1, 0, ..., 2, 0, 1],
       [1, 2, 1, ..., 0, 1, 0],
       ...,
       [0, 1, 1, ..., 3, 1, 0],
       [2, 0, 1, ..., 3, 0, 0],
       [2, 3, 1, ..., 0, 2, 0]])

In [150]:
poisson_samples = samples_counts_matrix * population
poisson_samples

array([[  0.        ,   0.        ,  51.79716109, ...,   0.        ,
         38.42408662,   0.        ],
       [ 56.18194637,  63.76595351,   0.        , ...,  91.57191842,
          0.        ,  61.23562377],
       [ 56.18194637, 127.53190702,  51.79716109, ...,   0.        ,
         38.42408662,   0.        ],
       ...,
       [  0.        ,  63.76595351,  51.79716109, ..., 137.35787763,
         38.42408662,   0.        ],
       [112.36389274,   0.        ,  51.79716109, ..., 137.35787763,
          0.        ,   0.        ],
       [112.36389274, 191.29786053,  51.79716109, ...,   0.        ,
         76.84817324,   0.        ]])

In [139]:
poisson_samples.mean(axis=1)

array([50.00719152, 50.36809465, 50.33363499, ..., 50.70434112,
       49.78765582, 50.18633115])

In [140]:
means_p = []
samples_counts = np.random.poisson(1, size*B)
samples_counts_matrix = samples_counts.reshape(B,-1)
poisson_samples = samples_counts_matrix * population
means_p = poisson_samples.mean(axis=1)
# for i in range(B):
#     each_value_count = np.random.poisson(1, size)
#     m = np.sum(population * each_value_count) / np.sum(each_value_count)
#     means_p.append(m)

In [141]:
means_b = []
for i in range(B):
    sample_boot = np.random.choice(population, replace=True, size=size)
    m = sample_boot.mean()
    means_b.append(m)

In [142]:
np.mean(means_p)

50.019365456693016

In [143]:
np.mean(means_b)

50.01467729252875

In [87]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

def estimate_first_type_error(df_pilot_group, df_control_group, metric_name, alpha=0.05, n_iter=10000, seed=None):
    """Оцениваем ошибку первого рода.

    Бутстрепим выборки из пилотной и контрольной групп тех же размеров, считаем долю случаев с значимыми отличиями.
    
    df_pilot_group - pd.DataFrame, датафрейм с данными пилотной группы
    df_control_group - pd.DataFrame, датафрейм с данными контрольной группы
    metric_name - str, названия столбца с метрикой
    alpha - float, уровень значимости для статтеста
    n_iter - int, кол-во итераций бутстрапа
    seed - int or None, состояние генератора случайных чисел.

    return - float, ошибка первого рода
    """
    
    pilot_metrics = df_pilot_group[metric_name]
    control_metrics = df_control_group[metric_name]
    equal_var = np.isclose(np.std(pilot_metrics), np.std(control_metrics), rtol=0.03)
    
    np.random.seed(seed)
    type_1_errors_count = 0
    
    for _ in range(n_iter):
        pilot_b = np.random.choice(pilot_metrics, replace=True, size=len(pilot_metrics))
        control_b = np.random.choice(control_metrics, replace=True, size=len(control_metrics))
        
        _, p_val = ttest_ind(pilot_b, control_b, equal_var=equal_var)
        type_1_errors_count += p_val < alpha
    
    return type_1_errors_count / n_iter
        
    # YOUR_CODE_HERE

In [90]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind


def estimate_second_type_error(df_pilot_group, df_control_group, metric_name, effects, alpha=0.05, n_iter=10000, seed=None):
    """Оцениваем ошибки второго рода.

    Бутстрепим выборки из пилотной и контрольной групп тех же размеров, добавляем эффект к пилотной группе,
    считаем долю случаев без значимых отличий.
    
    df_pilot_group - pd.DataFrame, датафрейм с данными пилотной группы
    df_control_group - pd.DataFrame, датафрейм с данными контрольной группы
    metric_name - str, названия столбца с метрикой
    effects - List[float], список размеров эффектов ([1.03] - увеличение на 3%).
    alpha - float, уровень значимости для статтеста
    n_iter - int, кол-во итераций бутстрапа
    seed - int or None, состояние генератора случайных чисел

    return - dict, {размер_эффекта: ошибка_второго_рода}
    """
    
    pilot_metrics = df_pilot_group[metric_name]
    control_metrics = df_control_group[metric_name]
    equal_var = np.isclose(np.std(pilot_metrics), np.std(control_metrics), rtol=0.03)
    
    np.random.seed(seed)
    type_2_errors = {}
    
    for effect in effects:
        type_2_errors[effect] = 0.
        for _ in range(n_iter):
            pilot_b = np.random.choice(pilot_metrics, replace=True, size=len(pilot_metrics))
            pilot_b *= effect
            control_b = np.random.choice(control_metrics, replace=True, size=len(control_metrics))

            _, p_val = ttest_ind(pilot_b, control_b, equal_var=equal_var)
            
            
            type_2_errors[effect] += p_val > alpha
        type_2_errors[effect] = type_2_errors[effect] / n_iter
    
    return type_2_errors

In [88]:
estimate_first_type_error(pd.DataFrame({'m': means_b}), pd.DataFrame({'m': means_p}), 'm')

0.2953

In [96]:
estimate_second_type_error(pd.DataFrame({'m': means_b}), pd.DataFrame({'m': means_b}), 'm', [1.00003, 0.98, 1.])

{1.00003: 0.0106, 0.98: 0.0, 1.0: 0.9529}

In [80]:
np.std(means_p)

0.007960523587459285

In [84]:
pilot_std = np.std(means_p)
control_std = np.std(means_b)
np.isclose(pilot_std, control_std, rtol=0.03)

True

In [82]:
pilot_std, control_std, pilot_std/control_std

(0.007960523587459285, 0.007856357662157582, 1.0132588064063641)

In [86]:
1 + True + False

2